In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.fft import fft
from scipy.optimize import curve_fit
import os

# Close all existing figures
plt.close('all')

which_bool = 'Front'
# Create necessary directories
plots_dir = which_bool + ' Plots'
data_dir = which_bool + ' Data'
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# List CSV files in the data directory
file_list = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
m = len(file_list)

# Color mapping for plotting
colors = plt.cm.hsv(np.linspace(0, 1, m))

# Initialize DataFrame to store peak results
peak_results = pd.DataFrame(columns=['File', 'Natural Frequency (kHz)', 'Amplitude', 'HWHM', 'Damping Coefficient'])

# Define the Lorentzian function without baseline parameter
def lorentzian(x, A, x0, gamma):
    return (A / np.pi) * (gamma / ((x - x0)**2 + gamma**2))

# Process each file
for ii in range(m):
    filename = file_list[ii]
    data = pd.read_csv(os.path.join(data_dir, filename))
    t = data.iloc[1200:, 0].to_numpy() * 1e3  # Time in seconds
    s = data.iloc[1200:, 1].to_numpy()  # Signal

    # Compute time increment and frequency range
    N = len(t)
    dt = np.mean(np.diff(t))
    freqs = np.fft.fftfreq(N, dt)[:N//2]  # Frequency in kHz

    # FFT computation and magnitude calculation
    FFT_result = fft(s)
    FFT_result_half = np.abs(FFT_result[:N//2]) * 2 / N

    # Determine frequency range based on file name
    if 'A' in filename:
        min_freq_threshold = 20  # kHz
        max_freq_threshold = 37.5  # kHz
    elif 'B' in filename:
        min_freq_threshold = 20  # kHz
        max_freq_threshold = 30  # kHz
    elif 'C' in filename:
        min_freq_threshold = 15  # kHz
        max_freq_threshold = 20  # kHz
    else:
        min_freq_threshold = 10  # Default minimum if unspecified
        max_freq_threshold = 39  # Default maximum if unspecified
        
    # Apply frequency limits
    valid_freqs = (freqs > min_freq_threshold) & (freqs < max_freq_threshold)
    freqs_limited = freqs[valid_freqs]
    FFT_result_half_limited = FFT_result_half[valid_freqs]

    if not freqs_limited.size:  # Check if the filtered frequencies array is empty
        print(f"No valid frequencies found in the range for file {filename}")
        continue

    # Determine a fixed baseline
    baseline = np.min(FFT_result_half_limited)

    # Find the natural frequency
    natural_freq_index = np.argmax(FFT_result_half_limited)
    natural_frequency = freqs_limited[natural_freq_index]

    plt.figure(figsize=(10, 6))
    plt.plot(freqs, FFT_result_half, color=colors[ii], marker='o', markersize=1)
    plt.scatter(natural_frequency, FFT_result_half_limited[natural_freq_index], color='red', label='Natural Frequency: {:.2f} kHz'.format(natural_frequency))
    plt.axhline(y=baseline, color='gray', linestyle='--', label='Baseline')
    plt.xlabel('Frequency (kHz)')
    plt.ylabel('Amplitude')
    title = 'FFT-' + filename.replace('.csv', '')
    plt.title(title)

    # Find peaks with stringent criteria within the valid frequency range
    peaks, properties = find_peaks(FFT_result_half_limited, height=np.max(FFT_result_half_limited) * 0.1, distance=20)
    peak_freqs = freqs_limited[peaks]

    # Find the peak closest to the natural frequency
    peak_distances = np.abs(peak_freqs - natural_frequency)
    closest_peak_idx = np.argmin(peak_distances)
    closest_peak_freq = peak_freqs[closest_peak_idx]

    # Fit Lorentzian only to the closest peak
    peak_width = 0.3
    peak_range = (freqs_limited > closest_peak_freq - peak_width) & (freqs_limited < closest_peak_freq + peak_width)
    # Adjusting initial guess for amplitude, possibly reducing it
    initial_amplitude = max(FFT_result_half_limited[peak_range] - baseline)
    initial_guess = [initial_amplitude, closest_peak_freq, 0.1]
    initial_guess = [FFT_result_half_limited[peaks][closest_peak_idx] - baseline, closest_peak_freq, 0.1]
    bounds = ([0, closest_peak_freq - 0.1, 0], [initial_amplitude, closest_peak_freq + 0.1, 1])  # Constrain amplitude and width
    
    try:
        popt, pcov = curve_fit(lorentzian, freqs_limited[peak_range], FFT_result_half_limited[peak_range] - baseline, p0=initial_guess, bounds=bounds)
        lorentzian_freqs = np.linspace(0.8*freqs_limited[peak_range][0], 1.2*freqs_limited[peak_range][0], 1000)  # Fine grid across the observed frequency range
        lorentzian_curve = lorentzian(lorentzian_freqs, *popt) + baseline  # Calculate the Lorentzian across this fine grid
        # plt.plot(lorentzian_freqs, lorentzian_curve, 'g', linewidth=2, label='Lorentzian Fit')    
        plt.plot(freqs_limited[peak_range], lorentzian(freqs_limited[peak_range], *popt) + baseline, color='green', label='Lorentzian Fit')
        # Store peak data including HWHM and Baseline
        new_row = pd.DataFrame({
            'File': [filename.replace('.csv', '')],
            'Natural Frequency (kHz)': [popt[1]],
            'Amplitude': [popt[0]],
            'HWHM': [popt[2]],
            'Damping Coefficient': [popt[2] / natural_frequency*1E3]
        })
        peak_results = pd.concat([peak_results, new_row], ignore_index=True)
    except RuntimeError:
        print("Fit failed for file", filename)

    plt.legend()
    plt.savefig(os.path.join(plots_dir, title + '_lorentzian_fit.png'), format='png')
    plt.show()

# Display results table
print(peak_results)

# Optionally, save the peak results to a CSV file
peak_results.to_csv(os.path.join(plots_dir, 'peak_analysis_results_lorentzian_natural_frequency.csv'), index=False)
